# Analyse interactive avec SparkR
## Initialisation
Utilisons Spark pour calculer certaines statistiques sur un jeu de données.

On doit dans un premier temps importer la librairie  R pour Spark

In [ ]:
library(SparkR)

ensuite récupérer le nom d'hôte du noeud maître

In [ ]:
hostname = system('hostname', intern=TRUE)
master_url = paste('spark://',hostname,':7077', sep='')

puis créer un contexte Spark et un context Spark SQL. Ce dernier permet de lire et manipuler des données structurées.

In [ ]:
sc <- sparkR.init(master_url)
sqlsc <- sparkRSQL.init(sc)

La création du contexte a démarré une nouvelle application SparkR. Pour s'en convaincre, on peut visiter la page de la console web de Spark :

[http://< hostname >:4040](http://< hostname >:4040)

## Création d'un jeu de données.

On va maintenant créer un jeu de données DataFrame à partir d'un fichier Parquet contenant les données de visite de Wikipedia. Les fichiers de données se trouvent dans le répertoire `data/pagecounts.parquet`.

In [ ]:
pagecounts <- read.df(sqlsc, 'data/pagecounts.parquet')

Le contenu d'un fichier pagecounts ressemble à ceci:
```
20090505-000000 af Spesiaal:Onlangse_wysigings 3 101681
20090505-000000 af Spesiaal:RecentChanges 2 2248
20090505-000000 af Suid-Afrika 1 30698
20090505-000000 af Tuisblad 14 155257 
20090505-000000 af Varkgriep 4 42236
20090505-000000 af Wikipedia 2 32796
```

Il s'agit d'un fichier tabulaire où la première colonne représente la date et l'heure d'échantillonnage, la deuxième colonne la langue, la troisième colonne le nom de la page, la quatrième colonne le nombre de visionnements de la page pour cette date, et la cinquième colonne la taille de la page en octet.

On peut le constater en utilisant la commande `take` du DataFrame pour obtenir les K premiers éléments d'un jeu de données. Ici `K = 10`.

In [ ]:
take(pagecounts, 10)

On peut vérifier la nature de l'objet retourné par la fonction `take`:

In [ ]:
class(take(pagecounts, 1))

La fonction take est donc une action retournant directement un résultat, soit un `data.frame` R.

## Mise en cache d'un DataFrame

Lorsque l'on s'attend à effectuer plusieurs opérations sur un même jeu de données, il peut être utile de spécifier à Spark de garder le DataFrame en mémoire, comme ceci:

In [ ]:
pagecounts <- cache(pagecounts)

Le jeu de données n'est transféré en mémoire que lorsqu'une action est appliquée. Les RDD stockés en mémoire peuvent être visualisés dans la section **Storage** de l'interface web de Spark
[http://< hostname > :4040/storage/](http://<hostname>:4040/storage/)

Pour libérer l'espace mémoire prise par un RDD en cache dont on n'aurait plus besoin, on appelle la méthode `unpersist`.

In [ ]:
unpersist(pagecounts)

## Action sur un jeu de données

La fonction `take` n'est qu'une parmi plusieurs *actions* que l'on peut effectuer sur un DataFrame. La liste exhaustive des actions est disponible à l'adresse suivante:
https://spark.apache.org/docs/latest/sql-programming-guide.html#dataframes

Prenons par exemple la fonction `count` qui retourne le nombre d'éléments dans un DataFrame.

In [ ]:
count(pagecounts)

Chaque action commise sur un DataFrame entraîne la création d'une ou de plusieurs tâches et ensuite la production d'un résultat. Toutes les tâches réalisées dans un même contexte Spark peuvent être visualisées dans la console web de Spark: [http://< hostname >:4040/](http://< hostname >:4040/)

Cette interface permet de suivre la progression d'une tâche et de consulter diverses métriques concernant l'exécution de la tâche, dont la durée de la tâche et les statistiques de cache.

## Information sur un DataFrame

On peut d'abord afficher simplement au format texte le contenu des 20 premières lignes en appelant la fonction  `showDF`.

In [ ]:
showDF(pagecounts)

On peut ensuite s'intéresser par exemple au schéma des données.

In [ ]:
printSchema(pagecounts)

La même information est aussi contenue dans la description de l'objet DataFrame de manière plus compact.

In [ ]:
print(pagecounts)

La fonction `head` retourne les 6 premières lignes du DataFrame, sous la forme d'un data.frame R. 

In [ ]:
head(pagecounts)

Vous constaterez qu'il s'agit du même résultat que si on avait appellé la fonction `take` avec 6 comme argument. 

In [ ]:
take(pagecounts, 6)

Tout comme R, SparkR permet d'effectuer plusieurs opérations identiques avec des appels de fonction différents. Il n'existe pas toujours de façon meilleure que les autres.

Si vous vous rendez sur la page : [http://< hostname >:4040/jobs/](http://< hostname >:4040/jobs/), vous constaterez que la plupart des appels de fonctions que l'on a effectué jusqu'à présent sont implémentés par le même appel natif.

## Filtrer un DataFrame

Comme on dispose maintenant d'un DataFrame facile à manipuler, on peut débuter l'analyse. Intéressons nous d'abord aux pages en langue anglaise.

La ligne suivante filtre le dernier DataFrame que nous avons créé et ne conserve que les entrées en français, c'est-à-dire les entrées dont la colonne "lang" correspond à la chaîne `"fr"`.

In [ ]:
pagecounts_fr <- filter(pagecounts, pagecounts$lang == "fr")

On peut aussi utiliser la fonction `where` qui a exactement la même signature et le même comportement.

In [ ]:
pagecounts_fr <- where(pagecounts, pagecounts$lang == "fr")
head(pagecounts_fr)

Nous allons vérifier la différence de la rapidité d'exécution avant et après avoir mis un DataFrame en cache.

In [ ]:
cache(pagecounts_fr)

Tant qu'aucune action n'a été commise sur le DataFrame, ce dernier ne sera pas mis en cache. Comme action, on peut, par exemple, pour compter le nombre de pages en français.

In [ ]:
ptm <- proc.time()
count(pagecounts_fr)
proc.time() - ptm

Puisqu'on a dit à Spark de conserver en mémoire ce nouveau jeu de données, on constatere que la vitesse d'exécution du décompte du nombre de pages est plus rapide à sa deuxième exécution.

In [ ]:
ptm <- proc.time()
count(pagecounts_fr)
proc.time() - ptm

## Opération d'aggrégation

On veut maintenant calculer le nombre total de page vue pour chaque langue. Pour ce faire, on utilise la fonction `groupBy`. Cette fonction permet de grouper tous les éléments partageant une même clé que l'on doit fournir.

Dans notre cas, la clé est la langue.

In [ ]:
pagegroups <- groupBy(pagecounts, pagecounts$lang)

La valeur qui nous intéresse est le nombre total de vues par langue.

In [ ]:
view_per_lang <- sum(pagecounts$pagecount)

On peut ensuite effectuer une aggrégation du nombre de vues des pages partageant la même langue

In [ ]:
lang_pagecounts <- agg(pagegroups, totalview = view_per_lang)

Puisque l'aggrégation est une transformation, le résultat est un nouveau RDD.

Pour visualiser la totalité du contenu de ce dernier RDD, on peut appeler la fonction `collect`.

In [ ]:
collect(lang_pagecounts)

Pour obtenir seulement les 5 langues les plus populaires, il faut effectuer un tri. Deux choix s'offrent à nous:

1- On effectue le tri localement. On crée d'abord un dataframe à partir du résultat:

In [ ]:
data <- collect(lang_pagecounts)

À partir du dataframe, on peut trier et obtenir les 5 langues les plus utilisées:

In [ ]:
top5_df = data[rev(order(data$totalview)),][1:5,]
top5_df

2- On utilise Spark pour effectuer le tri de manière distribuée en utilisant la commande `sortByKey`.

In [ ]:
top5 <- take(arrange(lang_pagecounts, desc(lang_pagecounts$totalview)), 5)

On peut finalement afficher un histogramme des 5 langues les plus utilisées.

In [ ]:
barplot(top5$totalview, names.arg=top5$lang)